# Fine-tune Bart on Wikihow

## Libraries and environment preparation

In [1]:
#Install essential packages
%%capture
!pip install datasets transformers rouge-score nltk wandb
!apt install git-lfs

In [2]:
#Colab Environment Check for GPU and RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

Sun Feb 13 18:11:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [3]:
# Make sure your version of Transformers is at least 4.11.0 
# to run the following code correctly:
import datasets
import transformers
print(transformers.__version__)

4.16.2


In [4]:
from transformers import AutoTokenizer    
# Huggingface Automodel class
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "facebook/bart-base"

In [5]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

## Loading the dataset

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# import dataset
raw_datasets = datasets.load_dataset("wikihow", "all", "/content/drive/MyDrive/dataset")

Downloading:   0%|          | 0.00/2.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Using custom data configuration all-6f5101161f12f62f


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikihow downloaded and prepared to /root/.cache/huggingface/datasets/wikihow/all-6f5101161f12f62f/1.2.0/5343fc81d685acaa086c9cc19eb8706206cd1f8b315792b04c1d7b92091c305e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'headline', 'title'],
        num_rows: 157252
    })
    validation: Dataset({
        features: ['text', 'headline', 'title'],
        num_rows: 5599
    })
    test: Dataset({
        features: ['text', 'headline', 'title'],
        num_rows: 5577
    })
})

## Preprocessing the data

In [9]:
# Import tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [10]:
# tokenlize inputs into map

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["headline"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
tokenized_datasets = raw_datasets.map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["text", "headline", "title"])

  0%|          | 0/158 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 157252
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5599
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5577
    })
})

## Fine-tuning the model

In [15]:
# Import tokenizer from model checkpoint and print detail
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [16]:
# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [17]:
# keep track with wandb
wandb.init(project="BART")

wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


Define `Seq2SeqTrainer` to compute the metrics from the predictions, and also do a bit of pre-processing to decode the predictions into texts:

In [18]:
# Define traing args, batch size and epoch
# batch size max 8 for input length 1024 on Colab Pro

batch_size = 4
epochs = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-wiki",
    load_best_model_at_end="eval_loss",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=1500, 
    save_steps=3000,
    eval_steps=3000,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
)

In [19]:
import nltk
import numpy as np
nltk.download('punkt')

metric = datasets.load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [20]:
# Pass into the trainer

train_dataset=tokenized_datasets["train"]
eval_dataset=tokenized_datasets["validation"]

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


We can now finetune our model by just calling the `train` method:

In [21]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 157252
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 39313
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
3000,2.459100,2.218038,24.010400,8.921300,20.157500,23.388800,19.830100
6000,2.323000,2.152289,24.737800,9.568900,20.867800,24.089400,19.788400
9000,2.281900,2.097617,25.224700,10.025700,21.396800,24.644400,19.765300
12000,2.218800,2.044122,25.820800,10.475200,21.883600,25.138400,19.778900
15000,2.196700,2.042885,25.792000,10.410600,21.861800,25.114100,19.787300
18000,2.187100,1.992621,25.894700,10.589700,21.935200,25.232600,19.788400
21000,2.111900,1.979864,26.607800,10.969100,22.508400,25.981300,19.783900
24000,2.080000,1.955184,26.096800,10.887200,22.240800,25.459300,19.791900
27000,2.081000,1.941329,26.369900,11.054500,22.414900,25.744000,19.793000
30000,2.062300,1.919969,26.549500,11.123000,22.553600,25.848200,19.807500


***** Running Evaluation *****
  Num examples = 5599
  Batch size = 4
Saving model checkpoint to bart-base-finetuned-xsum/checkpoint-3000
Configuration saved in bart-base-finetuned-xsum/checkpoint-3000/config.json
Model weights saved in bart-base-finetuned-xsum/checkpoint-3000/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-xsum/checkpoint-3000/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-xsum/checkpoint-3000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5599
  Batch size = 4
Saving model checkpoint to bart-base-finetuned-xsum/checkpoint-6000
Configuration saved in bart-base-finetuned-xsum/checkpoint-6000/config.json
Model weights saved in bart-base-finetuned-xsum/checkpoint-6000/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-xsum/checkpoint-6000/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-xsum/checkpoint-6000/special_tokens_map.json
***** Running Evaluation *****
 

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
3000,2.459100,2.218038,24.010400,8.921300,20.157500,23.388800,19.830100
6000,2.323000,2.152289,24.737800,9.568900,20.867800,24.089400,19.788400
9000,2.281900,2.097617,25.224700,10.025700,21.396800,24.644400,19.765300
12000,2.218800,2.044122,25.820800,10.475200,21.883600,25.138400,19.778900
15000,2.196700,2.042885,25.792000,10.410600,21.861800,25.114100,19.787300
18000,2.187100,1.992621,25.894700,10.589700,21.935200,25.232600,19.788400
21000,2.111900,1.979864,26.607800,10.969100,22.508400,25.981300,19.783900
24000,2.080000,1.955184,26.096800,10.887200,22.240800,25.459300,19.791900
27000,2.081000,1.941329,26.369900,11.054500,22.414900,25.744000,19.793000
30000,2.062300,1.919969,26.549500,11.123000,22.553600,25.848200,19.807500


***** Running Evaluation *****
  Num examples = 5599
  Batch size = 4
Saving model checkpoint to bart-base-finetuned-xsum/checkpoint-39000
Configuration saved in bart-base-finetuned-xsum/checkpoint-39000/config.json
Model weights saved in bart-base-finetuned-xsum/checkpoint-39000/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-xsum/checkpoint-39000/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-xsum/checkpoint-39000/special_tokens_map.json
Deleting older checkpoint [bart-base-finetuned-xsum/checkpoint-30000] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from bart-base-finetuned-xsum/checkpoint-39000 (score: 1.8871530294418335).


TrainOutput(global_step=39313, training_loss=2.1759241958170645, metrics={'train_runtime': 22332.5138, 'train_samples_per_second': 7.041, 'train_steps_per_second': 1.76, 'total_flos': 8.00378898149376e+16, 'train_loss': 2.1759241958170645, 'epoch': 1.0})

In [22]:
wandb.finish()

eval/gen_len,█▃▁▂▃▃▃▄▄▆▃▄▅
eval/loss,█▇▅▄▄▃▃▂▂▂▂▁▁
eval/rouge1,▁▃▄▆▆▆█▆▇████
eval/rouge2,▁▃▄▆▅▆▇▇▇████
eval/rougeL,▁▃▄▆▆▆▇▇▇▇███
eval/rougeLsum,▁▃▄▆▅▆█▆▇▇███
eval/runtime,▂▁▁▂▂▂▃▆▂▂▂▆█
eval/samples_per_second,▇██▇▇▇▆▃▇▇▇▃▁
eval/steps_per_second,▇██▇▇▇▆▃▇▇▇▃▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████


In [25]:
!ls -lh

total 1.5G
-rw-r--r-- 1 root root 1.5G Feb 14 00:53 bart-base-finetuned-wiki.zip
drwxr-xr-x 5 root root 4.0K Feb 14 00:40 bart-base-finetuned-xsum
drwx------ 5 root root 4.0K Feb 13 18:12 drive
drwxr-xr-x 1 root root 4.0K Feb  1 14:32 sample_data
drwxr-xr-x 3 root root 4.0K Feb 13 18:27 wandb


In [24]:
!zip -r /content/bart-base-finetuned-wiki.zip /content/bart-base-finetuned-xsum/checkpoint-39000/

  adding: content/bart-base-finetuned-xsum/checkpoint-39000/ (stored 0%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/special_tokens_map.json (deflated 50%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/vocab.json (deflated 59%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/training_args.bin (deflated 49%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/optimizer.pt (deflated 8%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/config.json (deflated 63%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/tokenizer.json (deflated 72%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/tokenizer_config.json (deflated 45%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/scheduler.pt (deflated 49%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/trainer_state.json (deflated 79%)
  adding: content/bart-base-finetuned-xsum/checkpoint-39000/pytorch_model.bin (deflated 8%)
  adding: cont

In [26]:
!cp bart-base-finetuned-wiki.zip '/content/drive/My Drive/weights/'

## Trying with a smaller dataset

In [ ]:
# Select to get smaller dataset
small_train = raw_datasets['train'].select(list(range(0, 5000)))
small_val = raw_datasets['validation'].select(list(range(0, 500)))
small_train

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 5000
})

In [ ]:
tokenized_train = small_train.map(preprocess_function, batched=True)
tokenized_val = small_val.map(preprocess_function, batched=True)
tokenized_train

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
    num_rows: 5000
})

In [ ]:
# Import original model in the cache
model_small = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.da0f3c0e2dc1c2fecc46738a1ebf4806f2fc36aae3d5c1947f21e063e7cab34b
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false

In [ ]:
# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_small)

In [ ]:
# Define traing args, batch size and epoch
# batch size max 16 on Colab Pro

batch_size = 6
epochs = 40
model_name = model_checkpoint.split("/")[-1]
args_small = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum-small",
    evaluation_strategy = "epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_strategy = "epoch",
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Pass into the trainer

train_dataset=tokenized_train
eval_dataset=tokenized_val

trainer_small = Seq2SeqTrainer(
    model_small,
    args_small,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
trainer_small.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running training *****
  Num examples = 5000
  Num Epochs = 40
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 33360
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.523100,2.151397,32.318100,12.072300,26.340100,26.362000,19.526000
2,2.023200,2.188214,32.980600,12.728400,26.707700,26.701200,19.748000
3,1.711800,2.232348,33.330200,12.352500,26.863400,26.867400,19.492000
4,1.385200,2.326119,33.195600,12.252300,26.479000,26.532400,19.806000
5,1.167600,2.458862,33.758200,12.415100,27.244200,27.258200,19.648000
6,0.987300,2.547485,32.516200,11.745500,26.000700,26.001400,19.728000
7,0.784900,2.675000,32.915300,12.152900,26.209500,26.217900,19.690000
8,0.651100,2.756314,31.950200,11.540300,25.757800,25.741600,19.784000
9,0.561200,2.836637,32.068000,11.816000,25.490300,25.460700,19.630000
10,0.431700,2.931043,31.934900,11.057100,25.669500,25.644000,19.880000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 6
Saving model checkpoint to bart-base-finetuned-xsum-small/checkpoint-834
Configuration saved in bart-base-finetuned-xsum-small/checkpoint-834/config.json
Model weights saved in bart-base-finetuned-xsum-small/checkpoint-834/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-xsum-small/checkpoint-834/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-xsum-small/checkpoint-834/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 6
Saving model checkpoint to bart-base-finetuned-xsum-small/checkpoint-1668
Configurati

KeyboardInterrupt: ignored

In [ ]:
wandb.finish()

eval/gen_len,▂▅▁▆▃▅▄▆▃▇█▆▇▅
eval/loss,▁▁▂▂▃▄▅▅▆▆▇▇██
eval/rouge1,▄▆▇▆█▄▆▃▃▃▃▃▁▄
eval/rouge2,▆█▇▇▇▅▆▄▅▃▄▃▁▄
eval/rougeL,▅▆▇▅█▄▄▃▂▂▂▁▁▄
eval/rougeLsum,▅▆▇▅█▄▄▃▂▂▂▁▁▄
eval/runtime,▅▆▆▇▄▄▅▅▄▆█▁▂▂
eval/samples_per_second,▄▃▃▂▅▄▄▄▅▃▁█▇▇
eval/steps_per_second,▄▃▃▂▅▅▄▅▅▃▁█▇▇
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
